In [253]:
import mod_sql
import pandas as pd
import imp
imp.reload(mod_sql)

<module 'mod_sql' from 'c:\\Users\\umjiwoo\\OneDrive\\문서\\금융빅데이터 과정\\ubion\\230418\\mod_sql\\__init__.py'>

In [254]:
# mod_sql을 통해 Database클래스 생성
_db = mod_sql.Database("localhost","root","1234","ubion7",3306)

In [255]:
sql = """
    select 
    `Country` as '국가', 
    `Units Sold` as '판매개수', 
    `Unit Price` as '가격', 
    `Unit Cost` as '원가', 
    (`Units Sold` * `Unit Price`) as '총수익', 
    (`Units Sold` * `Unit Cost`) as '총비용', 
    (`Units Sold` * `Unit Price` - `Units Sold` * `Unit Cost`) as '총이윤'
    from 
    `sales_records`
"""

In [256]:
df_sql = _db.sql_query(sql)

In [257]:
df_sql

,국가,판매개수,가격,원가,총수익,총비용,총이윤
0,Chad,4484,651.21,524.96,2920025.64,2353920.64,566105.00
1,Latvia,1075,47.45,31.79,51008.75,34174.25,16834.50
2,Pakistan,6515,154.06,90.93,1003700.90,592408.95,411291.95
3,Democratic Republic of the Congo,7683,668.27,502.54,5134318.41,3861014.82,1273303.59
4,Czech Republic,3491,47.45,31.79,165647.95,110978.89,54669.06
...,...,...,...,...,...,...,...
9995,Laos,8597,47.45,31.79,407927.65,273298.63,134629.02
9996,Liechtenstein,562,437.20,263.33,245706.40,147991.46,97714.94
9997,Democratic Republic of the Congo,2524,154.06,90.93,388847.44,229507.32,159340.12
9998,South Africa,8706,421.89,364.69,3672974.34,3174991.14,497983.20


In [258]:
df_pandas = pd.read_csv("../csv/Sales Records.csv")
df_pandas = df_pandas[["Country","Units Sold","Unit Price","Unit Cost"]]

In [259]:
df_pandas.columns = ['국가', '판매개수', '가격', '원가']

In [260]:
df_pandas["총수익"] = df_pandas["판매개수"] * df_pandas["가격"]

In [261]:
df_pandas["총비용"] = df_pandas["판매개수"] * df_pandas["원가"]

In [262]:
df_pandas["총이윤"] = df_pandas["총수익"] - df_pandas["총비용"]

In [263]:
df_pandas

,국가,판매개수,가격,원가,총수익,총비용,총이윤
0,Chad,4484,651.21,524.96,2920025.64,2353920.64,566105.00
1,Latvia,1075,47.45,31.79,51008.75,34174.25,16834.50
2,Pakistan,6515,154.06,90.93,1003700.90,592408.95,411291.95
3,Democratic Republic of the Congo,7683,668.27,502.54,5134318.41,3861014.82,1273303.59
4,Czech Republic,3491,47.45,31.79,165647.95,110978.89,54669.06
...,...,...,...,...,...,...,...
9995,Laos,8597,47.45,31.79,407927.65,273298.63,134629.02
9996,Liechtenstein,562,437.20,263.33,245706.40,147991.46,97714.94
9997,Democratic Republic of the Congo,2524,154.06,90.93,388847.44,229507.32,159340.12
9998,South Africa,8706,421.89,364.69,3672974.34,3174991.14,497983.20


### sql에서 값 반올림하기

In [264]:
sql = """
    select
    `Total Cost`,
    round(`Total Cost`,1)
    from
    sales_records
"""

_db.sql_query(sql)

,Total Cost,"round(`Total Cost`,1)"
0,2353920.64,2353920.6
1,34174.25,34174.2
2,592408.95,592409.0
3,3861014.82,3861014.8
4,110978.89,110978.9
...,...,...
9995,273298.63,273298.6
9996,147991.46,147991.5
9997,229507.32,229507.3
9998,3174991.14,3174991.1


### sql 그룹화
1. Country, Item Type을 기준으로 그룹화
2. Units Sold의 평균, 합계 생성
3. Country를 기준으로 오름차순 정렬

In [265]:
sql = """
    select
    `Country`,`Item Type`,
    AVG(`Units Sold`) as avg,
    SUM(`Units Sold`) as sum
    from `sales_records`
    group by `Country`,`Item Type`
    order by `Country`
"""

_db.sql_query(sql)

,Country,Item Type,avg,sum
0,Afghanistan,Baby Food,1368.0000,1368
1,Afghanistan,Beverages,4793.3333,14380
2,Afghanistan,Cereal,6101.8889,54917
3,Afghanistan,Clothes,5977.4000,29887
4,Afghanistan,Cosmetics,3044.2500,12177
...,...,...,...,...
2193,Zimbabwe,Meat,9640.0000,9640
2194,Zimbabwe,Office Supplies,4361.7500,17447
2195,Zimbabwe,Personal Care,9239.0000,18478
2196,Zimbabwe,Snacks,5438.0000,16314


In [266]:
df = pd.read_csv("../csv/Sales Records.csv")
df = df[["Country","Item Type","Units Sold"]]

In [267]:
df.groupby(["Country","Item Type"]).agg(["mean","sum"])["Units Sold"]

mean    sum
Country     Item Type                          
Afghanistan Baby Food        1368.000000   1368
            Beverages        4793.333333  14380
            Cereal           6101.888889  54917
            Clothes          5977.400000  29887
            Cosmetics        3044.250000  12177
...                                  ...    ...
Zimbabwe    Meat             9640.000000   9640
            Office Supplies  4361.750000  17447
            Personal Care    9239.000000  18478
            Snacks           5438.000000  16314
            Vegetables       6789.000000  33945

[2198 rows x 2 columns]

In [268]:
df.groupby(["Country","Item Type"])["Units Sold"].agg(["mean","sum"])

mean    sum
Country     Item Type                          
Afghanistan Baby Food        1368.000000   1368
            Beverages        4793.333333  14380
            Cereal           6101.888889  54917
            Clothes          5977.400000  29887
            Cosmetics        3044.250000  12177
...                                  ...    ...
Zimbabwe    Meat             9640.000000   9640
            Office Supplies  4361.750000  17447
            Personal Care    9239.000000  18478
            Snacks           5438.000000  16314
            Vegetables       6789.000000  33945

[2198 rows x 2 columns]

In [269]:
df.groupby(["Country","Item Type"]).agg(["mean","sum"])["Units Sold"].reset_index()

,Country,Item Type,mean,sum
0,Afghanistan,Baby Food,1368.000000,1368
1,Afghanistan,Beverages,4793.333333,14380
2,Afghanistan,Cereal,6101.888889,54917
3,Afghanistan,Clothes,5977.400000,29887
4,Afghanistan,Cosmetics,3044.250000,12177
...,...,...,...,...
2193,Zimbabwe,Meat,9640.000000,9640
2194,Zimbabwe,Office Supplies,4361.750000,17447
2195,Zimbabwe,Personal Care,9239.000000,18478
2196,Zimbabwe,Snacks,5438.000000,16314


### dept table에서 부서의 이름(dname)이 sales인 소속 사원의 정보 출력
1. dept테이블에서 부서이름이 sales인 deptno출력
2. emp 테이블에서 1번 과정에서 나온 deptno를 기준으로 사원 출력

In [270]:
sql = """
    select 
    `ename`
    from emp
    where `deptno`=(select 
            `deptno`
            from dept
            where `dname`="sales")
    """

_db.sql_query(sql)

,ename
0,ALLEN
1,WARD
2,MARTIN
3,BLAKE
4,TURNER
5,JAMES


In [271]:
sql = """
    select 
    *
    from emp
    where `deptno` in(
        select
        `deptno`
        from `dept`
        where `dname` in ("sales","research")
    )
"""

_db.sql_query(sql)

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20.0
1,7499.0,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30.0
2,7521.0,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30.0
3,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0
4,7654.0,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30.0
5,7698.0,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,0.0,30.0
6,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0
7,7844.0,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30.0
8,7876.0,ADAMS,CLERK,7788.0,1987-07-13,1100.0,0.0,20.0
9,7900.0,JAMES,CLERK,7698.0,1981-12-03,950.0,0.0,30.0


In [272]:
df_dept = pd.read_csv("../csv/dept.csv")
df_emp = pd.read_csv("../csv/emp.csv")

In [273]:
pd.concat([df_emp.loc[df_emp["DEPTNO"]==20],df_emp.loc[df_emp["DEPTNO"]==30]])

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20.0
3,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0
7,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0
10,7876.0,ADAMS,CLERK,7788.0,1987-07-13,1100.0,0.0,20.0
12,7902.0,FORD,ANALYST,7566.0,1981-12-03,3000.0,0.0,20.0
1,7499.0,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30.0
2,7521.0,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30.0
4,7654.0,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30.0
5,7698.0,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,0.0,30.0
9,7844.0,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30.0


In [274]:
# df_dept에서 부서이름이 SALES,RESEARCH인 데이터의 DEPTNO 속성값 가져오기 - 결과:Series형태

# df_dept.loc[(df_dept["DNAME"]=="SALES") | (df_dept["DNAME"]=="RESEARCH")]["DEPTNO"]

# df_dept.loc[(df_dept["DNAME"]=="SALES") | (df_dept["DNAME"]=="RESEARCH"),"DEPTNO"]

# df_dept.loc[(df_dept["DNAME"]=="SALES") | (df_dept["DNAME"]=="RESEARCH")]["DEPTNO"]

df_dept.loc[df_dept["DNAME"].isin(["RESEARCH","SALES"]), "DEPTNO"]

1    20.0
2    30.0
Name: DEPTNO, dtype: float64

In [275]:
# df_dept에서 부서이름이 SALES,RESEARCH인 데이터의 DEPTNO 속성값 가져와 리스트 형태로 변환
# isin()함수 사용을 위한 인자로 넣기 위함
deptno_list = df_dept.loc[df_dept["DNAME"].isin(["RESEARCH","SALES"]), "DEPTNO"].tolist()

In [276]:
df_emp.loc[df_emp["DEPTNO"].isin(deptno_list)]

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20.0
1,7499.0,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30.0
2,7521.0,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30.0
3,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0
4,7654.0,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30.0
5,7698.0,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,0.0,30.0
7,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0
9,7844.0,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30.0
10,7876.0,ADAMS,CLERK,7788.0,1987-07-13,1100.0,0.0,20.0
11,7900.0,JAMES,CLERK,7698.0,1981-12-03,950.0,0.0,30.0


#### 실습
1. emp데이터프레임에서 JOB컬럼의 데이터를 확인해 새로운 파생변수를 생성
2. job_code가 SALESMAN이면 1, MANAGER면 2, 나머지는 3으로 하여 새로운 파생변수에 대입

In [277]:
df_emp

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20.0
1,7499.0,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30.0
2,7521.0,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30.0
3,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0
4,7654.0,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30.0
5,7698.0,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,0.0,30.0
6,7782.0,CLARK,MANAGER,7839.0,1981-06-09,2450.0,0.0,10.0
7,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0
8,7839.0,KING,PRESIDENT,0.0,1981-11-17,5000.0,0.0,10.0
9,7844.0,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30.0


In [278]:
# JOB_CODE 파생변수 생성 & 0으로 초기화
df_emp["JOB_CODE"]=0

In [279]:
# 방법1)반복문 사용
for i in df_emp.index:
    if df_emp.loc[i,"JOB"]=="SALESMAN":
        df_emp.loc[i,"JOB_CODE"]=1
    elif df_emp.loc[i,"JOB"]=="MANAGER":
        df_emp.loc[i,"JOB_CODE"]=2
    else:
        df_emp.loc[i,"JOB_CODE"]=3

In [280]:
# 방법2)apply()사용-lambda
# df_emp["JOB"]값을 이용해 df_emp["JOB_CODE"] 파생변수의 값에 대응시키기 위해
# df_emp["JOB_CODE"]를 이용해 apply()함수 적용
# 여기서 x는 df_emp["JOB"] 각 행의 데이터
df_emp["JOB_CODE"] = df_emp["JOB"].apply(lambda 
                                            x:1 if (x=="SALESMAN") 
                                            else 2 if(x=="MANAGER") 
                                            else 3)
df_emp

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO,JOB_CODE
0,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20.0,3
1,7499.0,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30.0,1
2,7521.0,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30.0,1
3,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0,2
4,7654.0,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30.0,1
5,7698.0,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,0.0,30.0,2
6,7782.0,CLARK,MANAGER,7839.0,1981-06-09,2450.0,0.0,10.0,2
7,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0,3
8,7839.0,KING,PRESIDENT,0.0,1981-11-17,5000.0,0.0,10.0,3
9,7844.0,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30.0,1


In [281]:
# 방법3)apply()사용-change()만들어서 적용
def change(x):
    if x=="SALESMAN":
        res = 1
    elif x=="MANAGER":
        res = 2
    else:
        res = 3
    return res

df_emp["JOB_CODE"] = df_emp["JOB"].apply(change)

In [282]:
# 방법4)sql쿼리 직접 적용-case문
sql = """
    select
    `empno`,`ename`,`job`,
    case when `job` = "SALESMAN" then 1
        when `job` = "MANAGER" then 2
        else 3 end as `job_code`
    from emp
"""

_db.sql_query(sql)

,empno,ename,job,job_code
0,7369.0,SMITH,CLERK,3
1,7499.0,ALLEN,SALESMAN,1
2,7521.0,WARD,SALESMAN,1
3,7566.0,JONES,MANAGER,2
4,7654.0,MARTIN,SALESMAN,1
5,7698.0,BLAKE,MANAGER,2
6,7782.0,CLARK,MANAGER,2
7,7788.0,SCOTT,ANALYST,3
8,7839.0,KING,PRESIDENT,3
9,7844.0,TURNER,SALESMAN,1


In [283]:
# 방법5)sql쿼리 직접 적용-if문
sql = """
    select
    `empno`,`ename`,`job`,
    if(
        `job` = "salesman", 1,
        if(
            `job` = "manager",2,3
        )
    ) as `job_code`
    from emp
"""

# if(조건,참일때실행,거짓일때실행)
# if(`job` = "salesman", 1 , if(`job` = "manager",2,3)) as `job_code`
# 여기서는 거짓일 때에 해당하는 곳에 if문을 한 번 더 사용 ; job이 manager가 맞으면 2, 아니면 3이 할당되는 것

_db.sql_query(sql)

,empno,ename,job,job_code
0,7369.0,SMITH,CLERK,3
1,7499.0,ALLEN,SALESMAN,1
2,7521.0,WARD,SALESMAN,1
3,7566.0,JONES,MANAGER,2
4,7654.0,MARTIN,SALESMAN,1
5,7698.0,BLAKE,MANAGER,2
6,7782.0,CLARK,MANAGER,2
7,7788.0,SCOTT,ANALYST,3
8,7839.0,KING,PRESIDENT,3
9,7844.0,TURNER,SALESMAN,1


#### 실습2
1. emp데이터프레임에서 파생변수 H/L 생성
2. SAL이 2000이상이면 High, 아니면 Low로 데이터 삽입

In [284]:
df_emp["H/L"]=""

In [285]:
# 방법1)반복문 사용
for i in df_emp.index:
    if df_emp.loc[i,"SAL"] >= 2000:
        df_emp["H/L"] = "High"
    else:
        df_emp["H/L"] = "Low"

In [286]:
# 방법2)apply()함수 적용
df_emp["H/L"] = df_emp["SAL"].apply(lambda x:"High" if x>=2000 else "Low")

In [287]:
def change(x):
    if x>=2000:
        res = "High"
    else:
        res = "Low"

    return res

df_emp["H/L"] = df_emp["SAL"].apply(change)

In [288]:
#sql쿼리 직접 적용-case문
sql = """
    select
    `empno`,`ename`,`job`,`sal`,
    case when sal>=2000 then "High"
        else "Low" end as `H/L`
    from emp
"""

_db.sql_query(sql)

,empno,ename,job,sal,H/L
0,7369.0,SMITH,CLERK,800.0,Low
1,7499.0,ALLEN,SALESMAN,1600.0,Low
2,7521.0,WARD,SALESMAN,1250.0,Low
3,7566.0,JONES,MANAGER,2975.0,High
4,7654.0,MARTIN,SALESMAN,1250.0,Low
5,7698.0,BLAKE,MANAGER,2850.0,High
6,7782.0,CLARK,MANAGER,2450.0,High
7,7788.0,SCOTT,ANALYST,3000.0,High
8,7839.0,KING,PRESIDENT,5000.0,High
9,7844.0,TURNER,SALESMAN,1500.0,Low


In [289]:
#sql쿼리 직접 적용-if문
sql = """
    select
    `empno`,`ename`,`job`,`sal`,
    if(`sal`>=2000,"High","Low") as `H/L`
    from emp
"""

_db.sql_query(sql)

,empno,ename,job,sal,H/L
0,7369.0,SMITH,CLERK,800.0,Low
1,7499.0,ALLEN,SALESMAN,1600.0,Low
2,7521.0,WARD,SALESMAN,1250.0,Low
3,7566.0,JONES,MANAGER,2975.0,High
4,7654.0,MARTIN,SALESMAN,1250.0,Low
5,7698.0,BLAKE,MANAGER,2850.0,High
6,7782.0,CLARK,MANAGER,2450.0,High
7,7788.0,SCOTT,ANALYST,3000.0,High
8,7839.0,KING,PRESIDENT,5000.0,High
9,7844.0,TURNER,SALESMAN,1500.0,Low


In [290]:
df_emp

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO,JOB_CODE,H/L
0,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20.0,3,Low
1,7499.0,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30.0,1,Low
2,7521.0,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30.0,1,Low
3,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0,2,High
4,7654.0,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30.0,1,Low
5,7698.0,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,0.0,30.0,2,High
6,7782.0,CLARK,MANAGER,7839.0,1981-06-09,2450.0,0.0,10.0,2,High
7,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0,3,High
8,7839.0,KING,PRESIDENT,0.0,1981-11-17,5000.0,0.0,10.0,3,High
9,7844.0,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30.0,1,Low


### JOIN연산

In [291]:
sql = """
    select
    *
    from emp left join dept
    on emp.deptno = dept.deptno
"""

_db.sql_query(sql)

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO,dept.DEPTNO,DNAME,LOC
0,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20.0,20.0,RESEARCH,DALLAS
1,7499.0,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30.0,30.0,SALES,CHICAGO
2,7521.0,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30.0,30.0,SALES,CHICAGO
3,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0,20.0,RESEARCH,DALLAS
4,7654.0,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30.0,30.0,SALES,CHICAGO
5,7698.0,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,0.0,30.0,30.0,SALES,CHICAGO
6,7782.0,CLARK,MANAGER,7839.0,1981-06-09,2450.0,0.0,10.0,10.0,ACCOUNTING,NEW YORK
7,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0,20.0,RESEARCH,DALLAS
8,7839.0,KING,PRESIDENT,0.0,1981-11-17,5000.0,0.0,10.0,10.0,ACCOUNTING,NEW YORK
9,7844.0,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30.0,30.0,SALES,CHICAGO


In [292]:
sql = """
    select 
    emp.empno, emp.ename, emp.deptno, dept.dname, dept.loc
    from emp left join dept
    on emp.deptno=dept.deptno
"""

_db.sql_query(sql)

,empno,ename,deptno,dname,loc
0,7369.0,SMITH,20.0,RESEARCH,DALLAS
1,7499.0,ALLEN,30.0,SALES,CHICAGO
2,7521.0,WARD,30.0,SALES,CHICAGO
3,7566.0,JONES,20.0,RESEARCH,DALLAS
4,7654.0,MARTIN,30.0,SALES,CHICAGO
5,7698.0,BLAKE,30.0,SALES,CHICAGO
6,7782.0,CLARK,10.0,ACCOUNTING,NEW YORK
7,7788.0,SCOTT,20.0,RESEARCH,DALLAS
8,7839.0,KING,10.0,ACCOUNTING,NEW YORK
9,7844.0,TURNER,30.0,SALES,CHICAGO


In [293]:
pd.merge(df_emp,df_dept,how="right")

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO,JOB_CODE,H/L,DNAME,LOC
0,7782.0,CLARK,MANAGER,7839.0,1981-06-09,2450.0,0.0,10.0,2.0,High,ACCOUNTING,NEW YORK
1,7839.0,KING,PRESIDENT,0.0,1981-11-17,5000.0,0.0,10.0,3.0,High,ACCOUNTING,NEW YORK
2,7934.0,MILLER,CLERK,7782.0,1982-01-23,1300.0,0.0,10.0,3.0,Low,ACCOUNTING,NEW YORK
3,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20.0,3.0,Low,RESEARCH,DALLAS
4,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0,2.0,High,RESEARCH,DALLAS
5,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0,3.0,High,RESEARCH,DALLAS
6,7876.0,ADAMS,CLERK,7788.0,1987-07-13,1100.0,0.0,20.0,3.0,Low,RESEARCH,DALLAS
7,7902.0,FORD,ANALYST,7566.0,1981-12-03,3000.0,0.0,20.0,3.0,High,RESEARCH,DALLAS
8,7499.0,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30.0,1.0,Low,SALES,CHICAGO
9,7521.0,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30.0,1.0,Low,SALES,CHICAGO


In [294]:
# emp.deptno는 20또는30의 값만 가져서 dept와 right join 수행 시 dept에만 존재하는 deptno 값인 40에 해당하는 emp.deptno가 없어 Nan 반환
sql = """
    select 
    emp.empno, emp.ename, emp.deptno, dept.deptno, dept.dname, dept.loc
    from emp right join dept
    on emp.deptno=dept.deptno
"""

_db.sql_query(sql)

,empno,ename,deptno,dept.deptno,dname,loc
0,7934.0,MILLER,10.0,10.0,ACCOUNTING,NEW YORK
1,7839.0,KING,10.0,10.0,ACCOUNTING,NEW YORK
2,7782.0,CLARK,10.0,10.0,ACCOUNTING,NEW YORK
3,7902.0,FORD,20.0,20.0,RESEARCH,DALLAS
4,7876.0,ADAMS,20.0,20.0,RESEARCH,DALLAS
5,7788.0,SCOTT,20.0,20.0,RESEARCH,DALLAS
6,7566.0,JONES,20.0,20.0,RESEARCH,DALLAS
7,7369.0,SMITH,20.0,20.0,RESEARCH,DALLAS
8,7900.0,JAMES,30.0,30.0,SALES,CHICAGO
9,7844.0,TURNER,30.0,30.0,SALES,CHICAGO


### 문제
- 부서의 위치가 "NEW YORK","DALLAS"가 아닌 소속 사원들의 정보를 출력

In [295]:
sql = """
    select 
    * 
    from dept 
    where `loc` not in ("NEW YORK","DALLAS")
"""

_db.sql_query(sql)

,DEPTNO,DNAME,LOC
0,30.0,SALES,CHICAGO
1,40.0,OPERATIONS,BOSTON


In [296]:
sql = """
    select
    *
    from `emp`
    where `deptno` in (select 
                        `deptno` 
                        from dept 
                        where `loc` not in ("NEW YORK","DALLAS"))
"""

_db.sql_query(sql)

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7499.0,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30.0
1,7521.0,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30.0
2,7654.0,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30.0
3,7698.0,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,0.0,30.0
4,7844.0,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30.0
5,7900.0,JAMES,CLERK,7698.0,1981-12-03,950.0,0.0,30.0


In [297]:
sql = """
    select
    *
    from emp
    where deptno in (select deptno
                        from dept
                        where loc!="DALLAS" and loc!="NEW YORK")
"""

_db.sql_query(sql)

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7499.0,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30.0
1,7521.0,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30.0
2,7654.0,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30.0
3,7698.0,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,0.0,30.0
4,7844.0,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30.0
5,7900.0,JAMES,CLERK,7698.0,1981-12-03,950.0,0.0,30.0


In [298]:
sql = """
    select
    *
    from emp left join dept
    on emp.deptno = dept.deptno
    where loc not in ("NEW YORK","DALLAS")
"""

_db.sql_query(sql)

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO,dept.DEPTNO,DNAME,LOC
0,7499.0,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30.0,30.0,SALES,CHICAGO
1,7521.0,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30.0,30.0,SALES,CHICAGO
2,7654.0,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30.0,30.0,SALES,CHICAGO
3,7698.0,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,0.0,30.0,30.0,SALES,CHICAGO
4,7844.0,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30.0,30.0,SALES,CHICAGO
5,7900.0,JAMES,CLERK,7698.0,1981-12-03,950.0,0.0,30.0,30.0,SALES,CHICAGO


In [299]:
sql = """
    select * from tran1
    union
    select * from tran2
"""

_db.sql_query(sql)

,transaction_id,price,payment_date,customer_id
0,T0000000113,210000,2019-02-01 01:36:57,PL563502
1,T0000000114,50000,2019-02-01 01:37:23,HD678019
2,T0000000115,120000,2019-02-01 02:34:19,HD298120
3,T0000000116,210000,2019-02-01 02:47:23,IK452215
4,T0000000117,170000,2019-02-01 04:33:46,PL542865
...,...,...,...,...
6781,T0000006894,180000,2019-07-31 21:20:44,HI400734
6782,T0000006895,85000,2019-07-31 21:52:48,AS339451
6783,T0000006896,100000,2019-07-31 23:35:25,OA027325
6784,T0000006897,85000,2019-07-31 23:39:35,TS624738


In [300]:
sql = """
    select * from tran_d_1
    union
    select * from tran_d_2
"""

_db.sql_query(sql)

,detail_id,transaction_id,item_id,quantity
0,0,T0000000113,S005,1
1,1,T0000000114,S001,1
2,2,T0000000115,S003,1
3,3,T0000000116,S005,1
4,4,T0000000117,S002,2
...,...,...,...,...
7139,7139,T0000006894,S004,1
7140,7140,T0000006895,S002,1
7141,7141,T0000006896,S001,2
7142,7142,T0000006897,S002,1


In [301]:
sql = """
    select * from 
    tran1
    left join
    tran_d_1
    on tran1.transaction_id = tran_d_1.transaction_id
"""

_db.sql_query(sql)

,transaction_id,price,payment_date,customer_id,detail_id,tran_d_1.transaction_id,item_id,quantity
0,T0000000113,210000,2019-02-01 01:36:57,PL563502,0.0,T0000000113,S005,1.0
1,T0000000114,50000,2019-02-01 01:37:23,HD678019,1.0,T0000000114,S001,1.0
2,T0000000115,120000,2019-02-01 02:34:19,HD298120,2.0,T0000000115,S003,1.0
3,T0000000116,210000,2019-02-01 02:47:23,IK452215,3.0,T0000000116,S005,1.0
4,T0000000117,170000,2019-02-01 04:33:46,PL542865,4.0,T0000000117,S002,2.0
...,...,...,...,...,...,...,...,...
5238,T0000005108,210000,2019-06-15 02:42:41,HD315748,NaN,None,None,NaN
5239,T0000005109,150000,2019-06-15 03:36:16,HI215420,NaN,None,None,NaN
5240,T0000005110,50000,2019-06-15 03:44:06,IK880102,NaN,None,None,NaN
5241,T0000005111,210000,2019-06-15 04:14:06,IK074758,NaN,None,None,NaN


In [302]:
sql = """
    select * from 
        (select * from tran1 union select * from tran2) as tran
        left join
        (select * from tran_d_1 union select * from tran_d_2) as tran_d
        on tran.transaction_id = tran_d.transaction_id
"""

_db.sql_query(sql)

,transaction_id,price,payment_date,customer_id,detail_id,tran_d.transaction_id,item_id,quantity
0,T0000000113,210000,2019-02-01 01:36:57,PL563502,0,T0000000113,S005,1
1,T0000000114,50000,2019-02-01 01:37:23,HD678019,1,T0000000114,S001,1
2,T0000000115,120000,2019-02-01 02:34:19,HD298120,2,T0000000115,S003,1
3,T0000000116,210000,2019-02-01 02:47:23,IK452215,3,T0000000116,S005,1
4,T0000000117,170000,2019-02-01 04:33:46,PL542865,4,T0000000117,S002,2
...,...,...,...,...,...,...,...,...
7139,T0000006891,50000,2019-07-31 20:28:20,HI337854,7136,T0000006891,S001,1
7140,T0000006894,180000,2019-07-31 21:20:44,HI400734,7139,T0000006894,S004,1
7141,T0000006895,85000,2019-07-31 21:52:48,AS339451,7140,T0000006895,S002,1
7142,T0000006897,85000,2019-07-31 23:39:35,TS624738,7142,T0000006897,S002,1


In [303]:
df_tran1 = pd.read_csv("../csv/tran_1.csv")
df_tran2 = pd.read_csv("../csv/tran_2.csv")
df_tran_d_1 = pd.read_csv("../csv/tran_d_1.csv")
df_tran_d_2= pd.read_csv("../csv/tran_d_2.csv")

In [304]:
df_tran = pd.concat([df_tran1,df_tran2],axis=0)
df_tran_d = pd.concat([df_tran_d_1,df_tran_d_2],axis=0)
res_df = pd.merge(df_tran,df_tran_d,how="left",on="transaction_id")
res_df

,transaction_id,price,payment_date,customer_id,detail_id,item_id,quantity
0,T0000000113,210000,2019-02-01 01:36:57,PL563502,0,S005,1
1,T0000000114,50000,2019-02-01 01:37:23,HD678019,1,S001,1
2,T0000000115,120000,2019-02-01 02:34:19,HD298120,2,S003,1
3,T0000000116,210000,2019-02-01 02:47:23,IK452215,3,S005,1
4,T0000000117,170000,2019-02-01 04:33:46,PL542865,4,S002,2
...,...,...,...,...,...,...,...
7139,T0000006894,180000,2019-07-31 21:20:44,HI400734,7139,S004,1
7140,T0000006895,85000,2019-07-31 21:52:48,AS339451,7140,S002,1
7141,T0000006896,100000,2019-07-31 23:35:25,OA027325,7141,S001,2
7142,T0000006897,85000,2019-07-31 23:39:35,TS624738,7142,S002,1


In [305]:
sql = """
    select
    *
    from emp
    where ename like '%%S%%'
"""

_db.sql_query(sql)

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20.0
1,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0
2,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0
3,7876.0,ADAMS,CLERK,7788.0,1987-07-13,1100.0,0.0,20.0
4,7900.0,JAMES,CLERK,7698.0,1981-12-03,950.0,0.0,30.0


### 실습
1. pandas 이용해 emp.csv에서 ename이 s로 시작하는 데이터를 출력
2. s로 끝나는 데이터를 출력
3. s가 포함돼있는 데이터를 출력

In [306]:
data = pd.DataFrame()
for i in df_emp.index:
    if df_emp.loc[i,"ENAME"].startswith('S'):
        data = pd.concat([data,df_emp.loc[[i]]])

data

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO,JOB_CODE,H/L
0,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20.0,3,Low
7,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0,3,High


In [307]:
_list = []

for i in df_emp.index:
    if df_emp.loc[i,"ENAME"].startswith('S'):
        _list.append(i)

df_emp.loc[_list]


,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO,JOB_CODE,H/L
0,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20.0,3,Low
7,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0,3,High


In [308]:
# find('찾고자 하는 문자')함수 -> 문자 위치값(인덱스) 반환
# 찾고자 하는 문자를 포함하지 않는 경우 -1 반환
df_emp["ENAME"].str.find('S')

0     0
1    -1
2    -1
3     4
4    -1
5    -1
6    -1
7     0
8    -1
9    -1
10    4
11    4
12   -1
13   -1
Name: ENAME, dtype: int64

In [309]:
# ENAME이 'S'로 시작하는 사원 목록
df_emp.loc[df_emp["ENAME"].apply(lambda x:x.find('S')==0)]

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO,JOB_CODE,H/L
0,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20.0,3,Low
7,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0,3,High


In [310]:
# ENAME이 'S'로 끝나는 사원 목록
df_emp.loc[df_emp["ENAME"].apply(lambda x:x.find('S')==(len(x)-1))]

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO,JOB_CODE,H/L
3,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0,2,High
10,7876.0,ADAMS,CLERK,7788.0,1987-07-13,1100.0,0.0,20.0,3,Low
11,7900.0,JAMES,CLERK,7698.0,1981-12-03,950.0,0.0,30.0,3,Low


In [311]:
# ENAME에 'S'가 들어있는 모든 사원
# find('S')값이 -1 ==> 'S'를 포함하지 않음
# 따라서 find('S') != -1 는 'S'를 포함함을 의미
df_emp.loc[df_emp["ENAME"].apply(lambda x:x.find('S') != -1)]

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO,JOB_CODE,H/L
0,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20.0,3,Low
3,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0,2,High
7,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0,3,High
10,7876.0,ADAMS,CLERK,7788.0,1987-07-13,1100.0,0.0,20.0,3,Low
11,7900.0,JAMES,CLERK,7698.0,1981-12-03,950.0,0.0,30.0,3,Low


In [312]:
df_emp.loc[df_emp["ENAME"].apply(lambda x:'S' in x)]

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO,JOB_CODE,H/L
0,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20.0,3,Low
3,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0,2,High
7,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0,3,High
10,7876.0,ADAMS,CLERK,7788.0,1987-07-13,1100.0,0.0,20.0,3,Low
11,7900.0,JAMES,CLERK,7698.0,1981-12-03,950.0,0.0,30.0,3,Low
